## 一、项目简介
猫狗大战来源于Kaggle上的一个竞赛，利用给定的数据集，用算法实现猫和狗的识别。  

数据集由训练数据和测试数据组成：

    训练数据包含猫和狗各12500张图片；  
    测试数据包含12500张猫和狗的图片；
    
数据集下载链接：https://www.kaggle.com/c/dogs-vs-cats/data

## 二、项目流程：

### 1 数据准备

### 2 VGG16的Tensorflow实现
    定义功能函数
    定义VGG16模型类
### 3 VGG16模型复用
    微调  
    载入权重
### 4 数据输入
### 5 模型重新训练与保存
### 6 预测

## 三、代码示例

### 1、定义功能函数


In [2]:
# 卷积层
def conv(self, name, input_data, out_channel, trainable=False): # trainable参数变动
    in_channel = input_data.get_shape()[-1]
    with tf.variable_scope(name):
        kernel = tf.get_variable('weights', [3, 3, in_channel, out_channel], 
                                 dtype=tf.float32, trainable=False)
        biases = tf.get_variable('biases', [out_channel], 
                                 dtype=tf.float32, trainable=False)
        conv_res = tf.nn.conv2d(input_data, kernel, [1,1,1,1], padding='SAME')
        res = tf.nn.bias_add(conv_res, biases)
        out = tf.nn.relu(res, name=name)  
    self.parameters += [kernel, biases] # 将卷积层定义的参数加入列表
    return out

# 全连接层
def fc(self, name, input_data, out_channel, trainable=False):
    shape = input_data.get_shape().as_list()
    if len(shape)==4:
        size =  shape[-1] * shape[-2] * shape[-3]
    else:
        size=shape[1]
    input_data_flat = tf.reshape(input_data, [-1, size])
    with tf.variable_scope(name):
        weights = tf.get_variable(name='weights', shape=[size, out_channel], 
                                  dtype=tf.float32, trainable=False)
        biases = tf.get_variable(name='biases', shape=[out_channel],
                                 dtype=tf.float32, trainable=False)
        res = tf.matmul(input_data_flat, weights)
        out = tf.nn.relu(tf.nn.bias_add(res, biases))
    self.parameters += [weights, biases] # 将全连接层定义的参数加入列表    
    return out

def maxpool(self, name, input_data):
    out = tf.nn.max_pool(input_data, [1,2,2,1], [1,2,2,1], 
                         padding='SAME', name=name)
    return out    

### 2、定义VGG-16模型类

In [ ]:
class vgg16:
    def __init__(self, imgs):
        self.imgs = imgs
        self.convlayers()
        self.fc_layers()
        self.probs=self.fc8
        
    def fc_layers(self):
        self.fc6 = self.fc('fc1', self.pool5, 4096, trainable=False)
        self.fc7 = self.fc('fc2', self.fc6, 4096, trainable=False)
        self.fc8 = self.fc('fc3', self.fc7, n_class, trainable=True)
        # fc8是我们需要训练的trainable=True，2是n_class
        
    def convlayers(self):
        # conv1
        self.conv1_1 = self.conv('conv1re_1', self.imgs, 64, trainable=False)
        self.conv1_2 = self.conv('conv1_2', self.conv1_1, 64, trainable=False)
        self.pool1 = self.maxpool('poolre1', self.conv1_2)
        
        # conv2
        self.conv2_1 = self.conv('conv2_1', self.pool1, 128, trainable=False)
        self.conv2_2 = self.conv('conv2_2', self.conv2_1, 128, trainable=False)
        self.pool2 = self.maxpool('pool2', self.conv2_2)
        
        # conv3
        self.conv3_1 = self.conv('conv3_1', self.pool2, 256, trainable=False)
        self.conv3_2 = self.conv('conv3_2', self.conv3_1, 256, trainable=False)
        self.conv3_3 = self.conv('conv3_3', self.conv3_2, 256, trainable=False)
        self.pool3 = self.maxpool('pool3', self.conv3_3)
        
        # conv4
        self.conv4_1 = self.conv('conv4_1', self.pool3, 512, trainable=False)
        self.conv4_2 = self.conv('conv4_2', self.conv4_1, 512, trainable=False)
        self.conv4_3 = self.conv('conv4_3', self.conv4_2, 512, trainable=False)
        self.pool4 = self.conv('pool4', self.conv4_3)
        
        # conv5
        self.conv5_1 = self.conv('conv5_1', self.pool4, 512, trainable=False)
        self.conv5_2 = self.conv('conv5_2', self.conv5_1, 512, trainable=False)
        self.conv5_3 = self.conv('conv5_3', self.conv5_2, 512, trainable=False)
        self.pool5 = self.conv('pool5', self.conv5_3)
        
        
        # 载入权重
        def load_weights(self, weight_file, sess): # 这个函数将获取的权重载入VGG模型中
            weights = np.load(weight_file) 
            keys = sorted(weights.keys())
            for i, k in enumerate(keys):
                if i not in [30, 31]: # 剔除不需载入的层
                    sess.run(self.parameters[i].assign(weights[k]))
            print('-------------weights loaded-------------')
        

### 3、数据输入

In [ ]:
def get_file(file_dir):
    images = []
    temp = []
    for root, sub_folders, files in os.walk(file_dir):
        for name in files:
            images.append(os.path.join(root, name))
        for name in sub_folders:
            temp.append(os.path.join(root, name))
            labels = []
    for one_folder in temp:
        n_img = len(os.listdir(one_folder))
        letter = one_folder.split('/')[-1]
        if letter == 'cat':
            labels = np.append(labels, n_img * [0])
        else:
            labels = np.append(labels, n_img * [1])
            
    # shuffle
    temp = np.array([images, labels])
    temp = temp.transpose()
    np.random.shuffle(temp)
    image_list = list(temp[:,0])
    label_list = list(temp[:,1])
    label_list = [int(float(i)) for i in label_list]
    
    return image_list, label_list



In [4]:
from vgg_preprocess import preprocess_for_train

img_width = 224
img_height = 224

def get_batch(image_list, label_list, 
              img_width, img_height, 
              batch_size, capacity):# 通过读取列表来载入批量图片及标签
    image = tf.cast(image_list, tf.string)
    label = tf.cast(label_list, tf.int32)
    input_queue = tf.train.slice_input_producer([image, label])
    label = input_queue[1]
    image_contents = tf.read_file(input_queue[0])
    
    image = tf.image.decode_jpeg(image_contents, channels=3)
    image = preprocess_for_train(image, 224, 224)
    image_batch, label_batch = tf.train.batch([image, label], 
                                              batch_size=batch_size,
                                              num_threads=64,
                                              capacity=capacity)
    label_batch = tf.reshape(label_batch, [batch_size])
    
    return image_batch, label_batch



###  4、标签格式的重构

In [5]:
def onehot(labels):
    n_sample = len(labels)
    n_class = max(labels) + 1
    onehot_labels = np.zeros((n_sample, n_class))
    onehot_labels[np.arange(n_sample), labels] = 1
    return onehot_labels

### 5、模型重新训练与保存

In [6]:
import tensorflow as tf
import os
from time import time
import VGG16_model as model
import utils

In [7]:
startTime = time()
batch_size = 32
capacity = 256 # 内存中存储的最大数据容量
means = [123.68, 116.779, 103.939] # VGG训练时图像预处理所减均值（RGB三通道）

In [ ]:
xs, ys = utils.get_file('./data/train')# 获取图像列表与标签列表
# 通过读取列表来载入批量图片和标签
image_batch, label_batch = utils.get_batch(xs, ys, 224, 224, 
                                           batch_size, capacity)

x = tf.placeholder(tf.float32, [None, 224, 224, 3])
y = tf.placeholder(tf.int32, [None, 2])# 对猫和狗两个类别进行判定

vgg = model.vgg16(x)
fc8_finetuining = vgg.probs # 即softmax(fc8) # 微调
loss_function = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        logits=fc8_finetuining, labels=y))# 损失函数
# 优化器
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss_function)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 通过npz格式的文件获取VGG的相应权重参数，从而将权重注入即可实现复用
vgg.load_weights('./vgg16/vgg16_weights.npz', sess)
saver = tf.train.Saver()


In [ ]:
coord = tf.train.Coordinator() # 使用协调器Coordinator来管理线程
threads = tf.train.start_queue_runners(coord=coord,sess=sess)# 启动线程
epoch_start_time = time()

# 迭代训练
for i in range(1000):
    images, labels = sess.run([image_batch, label_batch])
    labels = utils.onehot(labels) # 用one-hot形式对标签进行编码
    
    sess.run(optimizer, feed_dict={x:images, y:labels})
    loss = sess.run(loss_function, feed_dict={x:images, y:labels})
    print('Now the loss is %f' % loss)
    
    epoch_end_time = time
    print('Current epoch takes:', (epoch_end_time - epoch_start_time))
    epoch_start_time = epoch_end_time
    
    if (i+1) % 500 == 0:
        saver.save(sess, os.path.join('./model/', 'epoch_{:06d}.ckpt'.format(i)))
        
    print('-----------Epoch %d is finished----------'% i)

# 模型保存
saver.save(sess, './model/')
print('Optimization Finished!')

# 关闭线程
coord.request_stop()# 通知其他线程关闭
coord.join(threads)# join操作等待其他线程结束，其他所有线程关闭之后，这一函数才能返回

### 6、预测

In [ ]:
means = [123.68, 116.779, 103.939]
x = tf.placeholder(tf.float32, [None, 224, 224, 3])

sess = tf.Session()
vgg = model.vgg16(x)
fc8_finetuining = vgg.probs

saver = tf.train.Saver()
print('Model restoring...')
saver.restore(sess, './model/')# 恢复最后保存的模型
# saver.restore(sess, './model/epoch_000800.ckpt')# 恢复指定检查点的模型

In [ ]:
filepath = './data/test/21.jpg' # 狗的图片
img = imread(filepath, mode='RGB')
img = imresize(img, (224, 224))
img = img.astype(np.float32)
for c in range(3):
    img[:,:,c] -= mean[c]
prob = sess.run(fc8_finetuining, feed_dict={x:[img]})
max_index = np.argmax(prob)

if max_index == 0:
    print('This is a cat with possibility %.6f' % prob[:,0])
else:
    print('This is a dog with possibility %.6f' % prob[:,1])